## RQ2 Text Analysis

General plan:
- Follow a similar approach to the Chai-Allah "Data Mining..." paper to create clusters from the data
    - Just use tokenisation first, if 2 big then use lemmatisation
    - for now, just use all words (don't worry about filtering for high frequency words)
    - Use a German version of the word2vec model (see what the baroni one is based on and try to find a German equivalent)
    - use k-means or ward's for the clustering

Open Questions
- How should I handle placenames? See if they form clusters? Filter using gazetteer?
- **HOW DO I CONNECT TO CLASS 3 AND 6 AREAS?** Do I run them separately (but then end up with different clusters for each) or do I run the model on everything to create a classifier and then run that on the two separate corpora? I think the latter makes more sense but I don't really understand how this would work.


Steps:
1. Initial Cleaning
2. Language Handling (skipped for now)
3. Tokenisation
4. Remove stop words and punctuation

In [22]:
# SETUP

# Import packages
import pandas as pd
import numpy as np
import spacy 


### Step 1: Initial Cleaning

General text preparation such as removing entries which are simply ['None'] and removing some special characters.


In [ ]:
# STEP 1: LOAD & CLEAN

# Load the master CSV from rq2_step1_data_collection
master = pd.read_csv("./processing/master.csv")


# Add ['None'] to any blank rows
# this is necessary for the next step, but then they will be removed later
master.fillna("['None']", inplace=True)

# Extract only the columns with text and combine them into a single column
raw_text = df = pd.DataFrame()
raw_text["all_text"] = master["description text"] + " " + master["photo_captions"] + " " + master["comments"]

# Now remove all the ['None'] text
raw_text["all_text"] = raw_text["all_text"].str.replace(r"\['None'\]", "", regex=True)

# Remove certain special characters: [, ], ', |
raw_text["all_text"] = raw_text["all_text"].str.replace(r"\[", "", regex=True)
raw_text["all_text"] = raw_text["all_text"].str.replace(r"\]", "", regex=True)
raw_text["all_text"] = raw_text["all_text"].str.replace(r"\'", "", regex=True)
raw_text["all_text"] = raw_text["all_text"].str.replace(r"\|", "", regex=True)

# Convert to list
raw_text = raw_text["all_text"].astype(str).values.tolist()

# Check
raw_text


### Step 2: Language Handling

- How should I handle the different languages? I will use German by default - should I remove English, Dutch?

**So far I have skipped this step and the different languages seem to be handled okay in the tokenisation step**

In [ ]:
# STEP 2: LANGUAGE

#skip for now

### Step 3: Tokenisation

German spacy model options: https://spacy.io/models/de 

de_core_news_sm (I've installed this one so far)
de_core_news_md
de_core_news_lg
de_dep_news_trf

Which is best to use?


In [ ]:
# STEP 3: TOKENISATION

# Load the spacy model
nlp = spacy.load("de_core_news_sm")

# Create an empty list to store the tokens
doc = []

# Tokenise the raw_text input
for string in raw_text:
    doc.extend(nlp(string))

# Print the tokens to check
#for token in doc:
#    print(token)
doc

[Finndorff-Blockland-St,
 .,
 Jürgen-Ritterhude-Findorff,
 Pause,
 am,
 Wümme,
 Deich,
 ,,
 Am,
 Wümme,
 Deich,
 ,,
 Weitere,
 Picknick,
 Möglichkeit,
 mit,
 Bank,
 ,,
 Foto,
 ,,
 Gastronomie,
 mit,
 WC,
 ,,
 Gastronomie,
 mit,
 WC,
 ,,
 Foto,
 ,,
 Weitere,
 Einkehr,
 Möglichkeit,
 mit,
 ECHTErrr,
 ,,
 Foto,
 ,,
 Kirche,
 St.,
 Jürgen,
 ,,
 Möglichkeit,
 für,
 Picknick,
 auf,
 der,
 Wiese,
 !,
 ,,
 Eiscafe,
 in,
 Ritterhude,
 ,,
 Wümme,
 Brücke,
 ,,
 Dammsiel,
 ,,
 Seitenwechsel,
 über,
 die,
 Wümme,
 Nice,
 trail,
 ,,
 nice,
 view,
 .,
 Flughafenrunde,
 Foto,
 ,,
 Foto,
 ,,
 Foto,
 ,,
 Abzweig,
 zum,
 Park,
 links,
 der,
 Weser,
 ,,
 ausgeschildert,
 !,
 !,
 ,,
 Abzweig,
 zum,
 Hotel,
 Robben,
 ,,
 Foto,
 ,,
 Foto,
 ,,
 Foto,
 ,,
 Ochtumdeich,
 ,,
 Fußweg,
 ,,
 ein,
 Stück,
 schieben,
 ,,
 wunderbarer,
 Weg,
 !,
 ,,
 Abzeig,
 zu,
 Picknickplatz,
 ,,
 Foto,
 ,,
 Foto,
 ,,
 Foto,
 ,,
 Badestrand,
 ,,
 Foto,
  ,
 OHZ,
 ,,
 Café,
 ,,
 Foto,
 WSV,
 Hasenbühren,
 -,
 Bremen,
 ,,
 Stephanieb

### Step 4: Stop words & punctuation

Remove stop words and punctuation from the token list